In [10]:
for i in range(5):
    print('testipynb')

testipynb
testipynb
testipynb
testipynb
testipynb


In [ ]:
from slack_sdk import WebClient
from load_dotenv import load_dotenv
from src.AiChallenge.config.configuration import SLACK_API
import os
load_dotenv()
slack_token = os.getenv('SLACK_API')


client = WebClient(token=slack_token)

def send_message(channel_id, message):
    response = client.chat_postMessage(
        channel=channel_id,
        text=message
    )
    if response.status_code == 200:
        print("Message sent successfully!")
    else:
        print("Message failed to send.")

channel_id = "compliance"
message = "Hi"

send_message(channel_id, message)

Message sent successfully!


In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List

def read_pdf(file_path: str) -> List[str]:
    try:
        loader = PyMuPDFLoader(file_path)
        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        
        text_chunks = [chunk.page_content for chunk in text_splitter.split_documents(documents)]
        return text_chunks
    except Exception as e:
        print(f"Error reading PDF: {str(e)}")
        raise

In [2]:
read_pdf('/home/ajay/Desktop/new/zania.ai/AI-Challenge/handbook.pdf')

Error reading PDF: Directory 'static/' does not exist


RuntimeError: Directory 'static/' does not exist

In [ ]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
import faiss
import numpy as np
embedding_dim = embedding_model.get_sentence_embedding_dimension()
faiss_index = faiss.IndexFlatL2(embedding_dim)
document_chunks = []
def process_pdf(file_path: str):
   
    global document_chunks
    pdf_reader = PdfReader(file_path)
    doc_text = "\n".join(page.extract_text() for page in pdf_reader.pages if page.extract_text())
    
    chunks = [doc_text[i:i+300] for i in range(0, len(doc_text), 300)]
    embeddings = embedding_model.encode(chunks)
    faiss_index.add(np.array(embeddings))
    document_chunks.extend(chunks)

/home/ajay/Desktop/new/zania.ai/AI-Challenge/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
process_pdf('/home/ajay/Desktop/new/zania.ai/AI-Challenge/handbook.pdf')

In [ ]:
from PyPDF2 import PdfReader
import re
def extract_text_from_pdf(pdf_file):
    reader = PdfReader(pdf_file)
    text = []
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text.append(clean_text(page_text))
    return "\n".join(text)
def clean_text(text):
    text = re.sub(r'\d+\.\d+\s+[A-Za-z\s]+\.*\.*\.*\d+', '', text)
    text = re.sub(r'\n{2,}', '\n', text)
    text = re.sub(r'\s{2,}', ' ', text)
    lines = text.splitlines()
    clean_lines = []
    for line in lines:
        if re.match(r'^\d{1,3}$', line.strip()): 
            continue
        if len(line.strip()) < 5: 
            continue
        clean_lines.append(line)
    return "\n".join(clean_lines)

In [3]:
s = extract_text_from_pdf('/home/ajay/Desktop/new/zania.ai/AI-Challenge/handbook.pdf')

In [5]:
print(clean_text(s))

Zania, Inc.
Zania Employee Handbook
September 07, 2023
TABLE OF CONTENTS
CORE POLICIES
1.2 At-Will Employment
5.0 PERFORMANCE, DISCIPLINE, LAYOFF, AND TERMINATION
5.1 Criminal Activity/Arrests
5.4 Open Door/Conflict Resolution Process
5.8 Post-Employment References
5.13 Workforce Reductions (Layoffs)
6.3 Employer-Provided Cell Phones/Mobile Devices
6.4 Nonsolicitation/Nondistribution Policy
6.5 Off-Duty Use of Employer Property or Premises
6.7 Personal Cell Phone/Mobile Device Use
7.3 Military Leave (USERRA)
7.4 Paid Time Off (PTO)
7.8 Workers' Compensation Insurance
CALIFORNIA POLICIES
HIRING AND ORIENTATION POLICIES
Accommodations for Victims of Crime or Abuse
Disability Accommodation
EEO Statement and Nonharassment Policy
Religious Accommodation
WAGE AND HOUR POLICIES
Accommodations for Nursing Mothers
Meal and Rest Periods
One Day Rest in Seven
Overtime
Pay Period
Reporting Time Pay
Travel Time Pay
Wage Disclosure Protection
Workday/Workweek
GENERAL POLICIES
Access to Personnel and

In [11]:
import faiss
import numpy as np

# FAISS index and document chunk list
embedding_dim = 384
faiss_index = faiss.IndexFlatL2(embedding_dim)
document_chunks = []

In [16]:
def chunk_text(text, max_chars=1000):
    
    sentences = re.split(r'(?<=\.)\s+', text)  # Split by sentence
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chars:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [18]:
print(chunk_text)

<function chunk_text at 0x7f772c96b560>


In [19]:
def add_document(doc):
   
    print(f"Document received: {doc}")
    chunks = chunk_text(doc.text, max_chars=1000)
    print(f"Chunks created: {chunks[:5]}...")  # Show only the first few chunks for brevity
    
    # Get the embeddings for the chunks
    embeddings = get_embeddings(chunks)
    print(f"Embeddings for chunks: {embeddings[:2]}...")  # Show only a sample for brevity
    
    # Add embeddings to FAISS index
    faiss_index.add(embeddings)
    
    # Store the document chunks for later retrieval
    document_chunks.extend(chunks)
    
    return {"message": "Document added successfully."}

In [27]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_embeddings(texts):
    """Get embeddings for a list of texts."""
    return embedding_model.encode(texts)

def retrieve_relevant_docs(query: str, top_k: int = 5):
    print(f"Query for retrieval: {query}")
    query_embedding = get_embeddings([query])
    print(f"Query embedding: {query_embedding}")
    
    _, indices = faiss_index.search(query_embedding, top_k)
    print(f"Top K document indices: {indices}")
    relevant_docs = [document_chunks[idx] for idx in indices[0]]
    print(f"Relevant document chunks: {relevant_docs[:2]}...")
    return [document_chunks[idx] for idx in indices[0]]

/home/ajay/Desktop/new/zania.ai/AI-Challenge/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [28]:
retrieve_relevant_docs(document_chunks)

Query for retrieval: []


IndexError: list index out of range

In [29]:
print(clean_text(s))

Zania, Inc.
Zania Employee Handbook
September 07, 2023
7.8 Workers' Compensation Insurance
CALIFORNIA POLICIES
HIRING AND ORIENTATION POLICIES
Accommodations for Victims of Crime or Abuse
Disability Accommodation
EEO Statement and Nonharassment Policy
Religious Accommodation
WAGE AND HOUR POLICIES
Accommodations for Nursing Mothers
Meal and Rest Periods
One Day Rest in Seven
Overtime
Pay Period
Reporting Time Pay
Travel Time Pay
Wage Disclosure Protection
Workday/Workweek
GENERAL POLICIES
Access to Personnel and Medical Records Files
Whistleblower Protections
BENEFITS
Crime Victim Leave
Disability Insurance
Jury Duty Leave
Paid Family Leave Insurance
Paid Sick Leave (Accrual Method)
Paid Sick Leave (Frontloading Method)
School Disciplinary Leave
Voting Leave
Witness Leave
SAFETY AND LOSS PREVENTION
Heat Illness Prevention
Workplace Smoking
TRADE SECRETS AND INVENTIONS
Inventions
CLOSING STATEMENT
ACKNOWLEDGMENT OF RECEIPT AND REVIEW
Core Policies
.1 A Welcome Policy
Welcome! You have j